Function Calling example

Load Azure OpenAI

In [32]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

Load Azure Cosmos DB

In [33]:
from azure.cosmos import CosmosClient
from dotenv import load_dotenv
import os
import json

load_dotenv()
# Initialize Cosmos Client
url = os.getenv('COSMOS_DB_ENDPOINT')
key = os.getenv('COSMOS_DB_KEY')
cosmosClient = CosmosClient(url, key)

# Select database
database_name = 'students'
database = cosmosClient.get_database_client(database_name)

# Select container
container_name = 'students'
container = database.get_container_client(container_name)

User Input

In [34]:
messages= [ {"role": "user", "content": "Find me student named Jose Rizal"} ]

Define Functions

In [35]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (administrator, ai-edge-engineer, ai-engineer, auditor, business-analyst, business-owner, business-user, data-analyst, data-engineer, data-scientist, database-administrator, developer, devops-engineer, functional-consultant, higher-ed-educator, identity-access-admin, ip-admin, k-12-educator, maker, network-engineer, parent-guardian, privacy-manager, risk-practitioner, school-leader, security-engineer, security-operations-analyst, service-adoption-specialist, solution-architect, startup-founder, student, support-engineer, technical-writer, technology-manager)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (dotnet, azure, bing, clarity, consumer, dynamics-365, entra, fabric, flip, github, hololens, industry-solutions, internet-explorer, intune, m365, microsoft-authentication-library, microsoft-edge, mem, ms-graph, makecode, power-platform, priva, office-teams, viva, microsoft-defender, microsoft-purview, minecraft, mrtk, ms-copilot, ms-website, nuance, office-365, bonsai, qdk, sql-server, sysinternals, vs, vs-app-center, vs-code, windows, xbox)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   },
   {
      "name":"search_exams",
      "description":"Retrieves exams from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the examiner (administrator, ai-edge-engineer, ai-engineer, auditor, business-analyst, business-owner, business-user, data-analyst, data-engineer, data-scientist, database-administrator, developer, devops-engineer, functional-consultant, higher-ed-educator, identity-access-admin, ip-admin, k-12-educator, maker, network-engineer, parent-guardian, privacy-manager, risk-practitioner, school-leader, security-engineer, security-operations-analyst, service-adoption-specialist, solution-architect, startup-founder, student, support-engineer, technical-writer, technology-manager)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (dotnet, azure, bing, clarity, consumer, dynamics-365, entra, fabric, flip, github, hololens, industry-solutions, internet-explorer, intune, m365, microsoft-authentication-library, microsoft-edge, mem, ms-graph, makecode, power-platform, priva, office-teams, viva, microsoft-defender, microsoft-purview, minecraft, mrtk, ms-copilot, ms-website, nuance, office-365, bonsai, qdk, sql-server, sysinternals, vs, vs-app-center, vs-code, windows, xbox)"
            }
         },
         "required":[]
      }
   },
   {
      "name":"search_student",
      "description":"Retrieves student based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "name":{
               "type":"string",
               "description":"The name of the student"},
            }
         },
         "required":[
             "name"
         ]
   }   
]

Making the function call. User input + Functions to be added in LLM

In [36]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

response_message = response.choices[0].message

print(response_message)
print(response_message.function_call.name)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "name": "Jose Rizal"\n}', name='search_student'), tool_calls=None)
search_student


Define function that we will call in the API

In [37]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)

def search_exams(role, product):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "type": "exams"
    }
    response = requests.get(url, params=params)
    modules = response.json()["exams"]
    results = []
    role = role.replace(' ', '-').lower()
    product = product.replace(' ', '-').lower()
    
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        if any(product in p for p in module["products"]) or any(role in r for r in module["roles"]):
            results.append({"title": title, "url": url})
    return str(results)



Find a specific student

In [41]:
def search_student(name):
    name=name
    query = f"SELECT * FROM c WHERE c.name = '{name}'"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    return json.dumps(items)

json.loads(response_message.function_call.arguments)


{'name': 'Jose Rizal'}

As a best practice, we will then see if the model wants to call a function.  After that, we will create one of the available functions and match it to the function that is being called. 
We will then take the arguments of the function and map them to arguments of from the LLM.

Lastly, we will append the function call message and the values that were returned by the `search_courses` message. This gives the LLM all the information it needs to
respond to the user using natural language. 

In [42]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name
    
    available_functions = {
            "search_courses": search_courses,
            "search_exams": search_exams,
            "search_student": search_student
    }
    function_to_call = available_functions[function_name] 
    
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )

    print(messages)



Recommended Function call:
search_student

Output of function call:
[{"id": "004", "name": "Jose Rizal", "course": "Humanities", "grade": 3.75, "sex": "M", "_rid": "rIAfAOuk+ioEAAAAAAAAAA==", "_self": "dbs/rIAfAA==/colls/rIAfAOuk+io=/docs/rIAfAOuk+ioEAAAAAAAAAA==/", "_etag": "\"04000f6c-0000-1800-0000-66057e9f0000\"", "_attachments": "attachments/", "_ts": 1711636127}]
<class 'str'>
[{'role': 'user', 'content': 'Find me student named Jose Rizal'}, {'role': 'assistant', 'function_call': {'name': 'search_student', 'arguments': '{\n  "name": "Jose Rizal"\n}'}, 'content': None}, {'role': 'function', 'name': 'search_student', 'content': '[{"id": "005", "name": "Teodora Agoncillo", "course": "Culinary Arts", "grade": 2.95, "sex": "F", "_rid": "rIAfAOuk+ioFAAAAAAAAAA==", "_self": "dbs/rIAfAA==/colls/rIAfAOuk+io=/docs/rIAfAOuk+ioFAAAAAAAAAA==/", "_etag": "\\"0400406c-0000-1800-0000-66057f9b0000\\"", "_attachments": "attachments/", "_ts": 1711636379}]'}, {'role': 'assistant', 'function_call': {

Now we will send the updated message to the LLM so we can receive a natural language response instead of an API JSON formatted response. 

In [43]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message.content)

Messages in next request:
[{'role': 'user', 'content': 'Find me student named Jose Rizal'}, {'role': 'assistant', 'function_call': {'name': 'search_student', 'arguments': '{\n  "name": "Jose Rizal"\n}'}, 'content': None}, {'role': 'function', 'name': 'search_student', 'content': '[{"id": "005", "name": "Teodora Agoncillo", "course": "Culinary Arts", "grade": 2.95, "sex": "F", "_rid": "rIAfAOuk+ioFAAAAAAAAAA==", "_self": "dbs/rIAfAA==/colls/rIAfAOuk+io=/docs/rIAfAOuk+ioFAAAAAAAAAA==/", "_etag": "\\"0400406c-0000-1800-0000-66057f9b0000\\"", "_attachments": "attachments/", "_ts": 1711636379}]'}, {'role': 'assistant', 'function_call': {'name': 'search_student', 'arguments': '{\n  "name": "Jose Rizal"\n}'}, 'content': None}, {'role': 'function', 'name': 'search_student', 'content': '[{"id": "004", "name": "Jose Rizal", "course": "Humanities", "grade": 3.75, "sex": "M", "_rid": "rIAfAOuk+ioEAAAAAAAAAA==", "_self": "dbs/rIAfAA==/colls/rIAfAOuk+io=/docs/rIAfAOuk+ioEAAAAAAAAAA==/", "_etag": "\\

: 